In [ ]:
import os
import rasterio
import cv2 as cv
import sys
import pandas as pd
from subprocess import run
import shlex

sys.path.insert(1, "../")
from utils import *

In [ ]:
downsampled_scenes = glob.glob("../data/inputs/landsat/true_color/**")
ref_image = downsampled_scenes[0]
tgt_images = downsampled_scenes[1:]

In [ ]:
outout_dir = "../data/outputs/KARIOS/L1GT_127111_Original"
os.makedirs(outout_dir, exist_ok=True)
log_file = "../data/outputs/KARIOS/L1GT_127111_Original/karios.log"
for tgt_image in tgt_images:
    cmd = f"python /home/ubuntu/Coreg/karios/karios/karios.py {tgt_image} {ref_image} --out {outout_dir} --log-file-path {log_file}"
    print(f"Running {cmd}")
    run(shlex.split(cmd))

In [ ]:
scene_names = []
shifts = []
with open(log_file, "r") as f:
    for line in f:
        if "_T2_TC.TIF" in line:
            scene_names.append(line.strip().split(" ")[-1])
        if "DX/DY(KLT) MEAN" in line:
            splits = line.strip().split(" ")
            shifts.append([float(splits[-3]), float(splits[-1])])

shifts_dict = {}
for f, sh in zip(scene_names, shifts):
    shifts_dict[f] = sh

In [ ]:
output_dir = "../data/outputs/KARIOS/L1GT_127111_Original/Aligned"
os.makedirs(output_dir, exist_ok=True)
for key in list(shifts_dict.keys()):
    output_path = os.path.join(output_dir, os.path.basename(key))
    shift_x, shift_y = shifts_dict[key]
    tgt_aligned = warp_affine_dataset(
        key, output_path, translation_x=shift_x, translation_y=shift_y
    )

In [ ]:
processed_output_images = glob.glob("../data/outputs/KARIOS/L1GT_127111_Original/Aligned/**")
ref_img = cv.cvtColor(
    flip_img(rasterio.open(ref_image).read()), cv.COLOR_BGR2GRAY
).astype("uint8")
tgt_aligned_list = []
for tgt_image in processed_output_images:
    tgt_aligned_list.append(
        cv.cvtColor(
            flip_img(rasterio.open(tgt_image).read()), cv.COLOR_BGR2GRAY
        ).astype("uint8")
    )

datasets_paths = [ref_image] + processed_output_images
ssims_aligned = [
    np.round(ssim(ref_img, tgt, win_size=3), 3) for tgt in tgt_aligned_list
]
mse_aligned = [np.round(mse(ref_img, tgt), 3) for tgt in tgt_aligned_list]
target_titles = [f"target_{str(i)}" for i in range(len(tgt_aligned_list))]
datasets_titles = ["Reference"] + [
    f"{target_title}, ssim:{ssim_score}, mse:{mse_score}"
    for target_title, ssim_score, mse_score in zip(
        target_titles, ssims_aligned, mse_aligned
    )
]
make_difference_gif(
    datasets_paths,
    "../data/outputs/KARIOS/L1GT_127111_Original/karios.gif",
    datasets_titles,
    fps=3,
)

datasets_paths = [ref_image] + tgt_images
tgt_raw_list = []
for tgt_image in tgt_images:
    tgt_raw_list.append(
        cv.cvtColor(
            flip_img(rasterio.open(tgt_image).read()), cv.COLOR_BGR2GRAY
        ).astype("uint8")
    )
ssims_aligned_raw = [
    np.round(ssim(ref_img, tgt, win_size=3), 3) for tgt in tgt_raw_list
]
mse_aligned_raw = [np.round(mse(ref_img, tgt), 3) for tgt in tgt_raw_list]
target_titles = [f"target_{str(i)}" for i in range(len(tgt_raw_list))]
datasets_titles = ["Reference"] + [
    f"{target_title}, ssim:{ssim_score}, mse:{mse_score}"
    for target_title, ssim_score, mse_score in zip(
        target_titles, ssims_aligned_raw, mse_aligned_raw
    )
]
make_difference_gif(
    datasets_paths,
    "../data/outputs/KARIOS/L1GT_127111_Original/karios_raw.gif",
    datasets_titles,
    fps=3,
)

out_ssim = "../data/outputs/KARIOS/L1GT_127111_Original/karios.csv"
out_ssim_df = pd.DataFrame(
    zip(target_titles, ssims_aligned_raw, ssims_aligned, mse_aligned_raw, mse_aligned),
    columns=["Title", "SSIM Raw", "SSIM Aligned", "MSE Raw", "MSE Aligned"],
    index=None,
            )
out_ssim_df.to_csv(out_ssim, encoding="utf-8")